# RNN 기본예제

* 일부 코드 [김성훈 교수님 TensorFlow 강의자료](https://github.com/hunkim/DeepLearningZeroToAll) 참고

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

import tensorflow as tf

slim = tf.contrib.slim
rnn = tf.contrib.rnn

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

![image](https://cloud.githubusercontent.com/assets/901975/23348727/cc981856-fce7-11e6-83ea-4b187473466b.png)

## Basic RNN Cell

In [3]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('one_cell', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2
    cell = rnn.BasicRNNCell(num_units=hidden_size)
    print("output size: {}, state size: {}".format(cell.output_size, cell.state_size))

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    print("x_data: {}".format(x_data))
    outputs, state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: {}".format(outputs_))
    print("state: {}".format(state_))

output size: 2, state size: 2
x_data: [[[1. 0. 0. 0.]]]
outputs: [[[-0.55518246 -0.5669185 ]]]
state: [[-0.55518246 -0.5669185 ]]


![image](https://cloud.githubusercontent.com/assets/901975/23383634/649efd0a-fd82-11e6-925d-8041242743b0.png)

In [4]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('add_sequances', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = rnn.BasicRNNCell(num_units=hidden_size)

    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print("x_data shape: {}".format(x_data.shape))
    print("x_data: {}".format(x_data))
    outputs, state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: \n{}".format(outputs_))
    print("state: {}".format(state_))

x_data shape: (1, 5, 4)
x_data: [[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]]
outputs: 
[[[ 0.2502348   0.6181834 ]
  [ 0.55891544  0.2544017 ]
  [-0.42626688  0.49625376]
  [ 0.05493243  0.7646796 ]
  [ 0.73394966 -0.3102551 ]]]
state: [[ 0.73394966 -0.3102551 ]]


## [`tf.nn.static_rnn`](https://www.tensorflow.org/api_docs/python/tf/nn/static_rnn)

* `tf.nn.static_rnn` low level code

```python
state = cell.zero_state(...)
outputs = []
for input_ in inputs:
  output, state = cell(input_, state)
  outputs.append(output)
return (outputs, state)
```

![image](https://cloud.githubusercontent.com/assets/901975/23383681/9943a9fc-fd82-11e6-8121-bd187994e249.png)

## Basic LSTM Cell

In [5]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('3_batches_LSTM', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: \n{}\n".format(outputs_))
    #print("memory cell state: \n{}".format(state_[0])) # print memory cell
    #print("hidden cell state: \n{}".format(state_[1])) # print hidden state
    print("memory cell state: \n{}".format(state_.c)) # print memory cell
    print("hidden cell state: \n{}".format(state_.h)) # print hidden state

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
outputs: 
[[[-0.04200047 -0.07450665]
  [ 0.00470946 -0.04139847]
  [-0.02686612  0.00093464]
  [-0.04474749  0.0348079 ]
  [-0.04594119 -0.06795195]]

 [[ 0.04210941  0.0022138 ]
  [ 0.01712591 -0.10112297]
  [-0.02285778 -0.03712514]
  [-0.04265621  0.00582262]
  [-0.05383435  0.03949804]]

 [[-0.02910151  0.03234435]
  [-0.04585794  0.05841699]
  [ 0.00604393  0.05195517]
  [ 0.04733184  0.04134284]
  [-0.00534663  0.06012887]]]

memory cell state: 
[[-0.0788601  -0.12787348]
 [-0.13339275  0.08232088]
 [-0.01353076  0.12351479]]
hidden cell state: 
[[-0.04594119 -0.06795195]
 [-0.05383435  0.03949804]
 [-0.00534663  0.06012887]]


## GRU Cell

In [6]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('3_batches_GRU', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

    hidden_size = 2
    cell = rnn.GRUCell(num_units=hidden_size)
    outputs, state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: \n{}\n".format(outputs_))
    print("hidden cell state: \n{}".format(state_)) # print hidden state

outputs: 
[[[-0.2626353  -0.10113645]
  [ 0.05246437 -0.09292916]
  [ 0.10953575  0.1032918 ]
  [ 0.17181712  0.21941169]
  [ 0.03649883  0.28476456]]

 [[ 0.246008   -0.01989472]
  [ 0.06693815  0.09941221]
  [ 0.13537657  0.22427194]
  [ 0.2024049   0.2932141 ]
  [ 0.26339865  0.32779714]]

 [[ 0.07086135  0.17298564]
  [ 0.14426777  0.2706858 ]
  [ 0.38182002  0.21391447]
  [ 0.5190277   0.15779868]
  [ 0.52004856  0.19371179]]]

hidden cell state: 
[[0.03649883 0.28476456]
 [0.26339865 0.32779714]
 [0.52004856 0.19371179]]


### Dynamic length

In [7]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('3_batches_dynamic_length', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, state = tf.nn.dynamic_rnn(cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: \n{}\n".format(outputs_))
    print("memory cell state: \n{}".format(state_.c)) # print memory cell
    print("hidden cell state: \n{}".format(state_.h)) # print hidden state

outputs: 
[[[ 0.11871063  0.05536252]
  [ 0.02785934  0.05481167]
  [-0.06707699  0.00771322]
  [-0.1609556  -0.05433674]
  [-0.25996542  0.04671576]]

 [[-0.03984005 -0.01051588]
  [-0.16625214  0.06778486]
  [-0.22376214  0.04054031]
  [ 0.          0.        ]
  [ 0.          0.        ]]

 [[-0.10502876 -0.0526636 ]
  [-0.19158715 -0.11802494]
  [-0.14602473 -0.1404725 ]
  [-0.16414692 -0.15892558]
  [ 0.          0.        ]]]

memory cell state: 
[[-0.42165747  0.12546143]
 [-0.41787526  0.06052474]
 [-0.5189897  -0.25719088]]
hidden cell state: 
[[-0.25996542  0.04671576]
 [-0.22376214  0.04054031]
 [-0.16414692 -0.15892558]]


### Feed initial state

In [8]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('initial_state', reuse=tf.AUTO_REUSE) as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)

    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size=2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, state = tf.nn.dynamic_rnn(cell, x_data,
                                       initial_state=initial_state,
                                       dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: \n{}\n".format(outputs_))
    print("memory cell state: \n{}".format(state_.c)) # print memory cell
    print("hidden cell state: \n{}".format(state_.h)) # print hidden state

outputs: 
[[[-0.0923196  -0.05381259]
  [-0.17942934  0.02550524]
  [-0.1008595  -0.02589609]
  [-0.06396154 -0.07645492]
  [ 0.04206917 -0.15921205]]

 [[-0.09433518  0.06279369]
  [ 0.02118034 -0.07592113]
  [ 0.01709709 -0.13287519]
  [ 0.01941351 -0.16250803]
  [ 0.02178395 -0.18334514]]

 [[ 0.00457918 -0.06666386]
  [ 0.00937748 -0.11540769]
  [-0.08648608 -0.03240742]
  [-0.15848157  0.03932928]
  [-0.08878244 -0.01559131]]]

memory cell state: 
[[ 0.0682736  -0.31723994]
 [ 0.046914   -0.30664766]
 [-0.19359842 -0.02521864]]
hidden cell state: 
[[ 0.04206917 -0.15921205]
 [ 0.02178395 -0.18334514]
 [-0.08878244 -0.01559131]]


## Create new input data

In [9]:
# Create new input data
batch_size=3
sequence_length=5
input_dim=3

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
print(x_data)  # [batch, sequence_length, input_dim]

[[[ 0.  1.  2.]
  [ 3.  4.  5.]
  [ 6.  7.  8.]
  [ 9. 10. 11.]
  [12. 13. 14.]]

 [[15. 16. 17.]
  [18. 19. 20.]
  [21. 22. 23.]
  [24. 25. 26.]
  [27. 28. 29.]]

 [[30. 31. 32.]
  [33. 34. 35.]
  [36. 37. 38.]
  [39. 40. 41.]
  [42. 43. 44.]]]


## Bi-directional RNN

![bidirectional_rnn](https://user-images.githubusercontent.com/11681225/46912324-3d53e400-cfad-11e8-8b09-85d8ebdb0e66.png)

### Bi-directional RNN with basic RNN Cell

In [10]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('bi-directional_RNN', reuse=tf.AUTO_REUSE) as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicRNNCell(num_units=4)
    cell_bw = rnn.BasicRNNCell(num_units=4)

    # x_data.shape = (3, 5, 3) [batch, sequence_length, input_dim]
    # outputs.shape = two element tuple of (3, 5, 4) [batch, sequence_length, input_dim] shape
        # outputs[0]: cell_fw, outputs[1]: cell_bw
    # state.shape = two element tuple (3, 4) [batch, sequence_length, input_dim]
        # states[0]: cell_fw, states[1]: cell_bw
    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, states_ = sess.run([outputs, states])
    print("foward outputs: shape {}\n{}\n".format(outputs_[0].shape, outputs_[0]))
    print("forward hidden cell state: \n{}\n".format(states_[0]))
    print("backward outputs: shape {}\n{}\n".format(outputs_[1].shape, outputs_[1]))
    print("backward hidden cell state: \n{}".format(states_[1]))

foward outputs: shape (3, 5, 4)
[[[-0.372043   -0.05765534  0.49066675  0.5124193 ]
  [-0.19166756 -0.6246104   0.42906988  0.6761464 ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 0.9890785  -0.9969672   0.66415185  0.85173994]
  [ 0.97311914 -0.9996541  -0.23717575  0.8176195 ]
  [ 0.995216   -0.99970293 -0.2903907   0.6522441 ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 0.9999724  -0.99999475  0.7869898   0.9610064 ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]]

forward hidden cell state: 
[[-0.19166756 -0.6246104   0.42906988  0.6761464 ]
 [ 0.995216   -0.99970293 -0.2903907   0.6522441 ]
 [ 0.9999724  -0.99999475  0.7869898   0.9610064 ]]

### Bi-directional RNN with basic LSTM Cell

In [11]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('bi-directional_LSTM', reuse=tf.AUTO_REUSE) as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicLSTMCell(num_units=4, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=4, state_is_tuple=True)

    # x_data.shape = (3, 5, 3) [batch, sequence_length, input_dim]
    # outputs.shape = two element tuple of (3, 5, 4) [batch, sequence_length, input_dim] shape
        # outputs[0]: cell_fw, outputs[1]: cell_bw
    # state.shape = two element tuple (3, 4) [batch, sequence_length, input_dim]
        # states[0]: cell_fw, states[1]: cell_bw
    outputs, state = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("foward outputs: shape: {}\n{}\n".format(outputs_[0].shape, outputs_[0]))
    print("forward memory cell state: \n{}".format(state_[0].c))
    print("forward hidden cell state: \n{}\n".format(state_[0].h))
    print("backward outputs: shape: {}\n{}\n".format(outputs_[1].shape, outputs_[1]))
    print("backward memory cell state: \n{}".format(state_[1].c))
    print("backward hidden cell state: \n{}".format(state_[1].h))

foward outputs: shape: (3, 5, 4)
[[[ 2.6394698e-01  1.2007784e-01 -2.1567455e-01 -1.0304569e-01]
  [ 4.2820162e-01  4.9870023e-01 -6.6084749e-01 -3.0921070e-02]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]

 [[ 6.0132229e-01  7.0859015e-01 -7.6147884e-01 -9.8406803e-09]
  [ 4.6547440e-01  8.6408085e-01 -7.6157928e-01 -2.7965594e-09]
  [ 4.8547259e-01  8.9369202e-01 -7.6159167e-01 -5.1230581e-10]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]

 [[ 6.6101509e-01  7.5673598e-01 -7.6159418e-01 -3.0194975e-16]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]]



## Multi-Layer RNN

![multilayer_rnn](https://user-images.githubusercontent.com/11681225/46912330-5fe5fd00-cfad-11e8-95c2-94bb8e7b1bf6.png)

### Multi-Layer RNN with basic RNN Cell

In [12]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('MultiRNN_RNN', reuse=tf.AUTO_REUSE) as scope:
    # MultiLayer RNN
    num_layers = 3
    multi_cells = rnn.MultiRNNCell([rnn.BasicRNNCell(4) for _ in range(num_layers)])

    outputs, states = tf.nn.dynamic_rnn(multi_cells, x_data,
                                        sequence_length=[2, 3, 1],
                                        dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, states_ = sess.run([outputs, states])
    print("outputs: shape: {}\n{}\n".format(outputs_.shape, outputs_))
    print("Number of Layers: {}".format(len(states_))) 
    for i in range(num_layers):
      print("Layer {} hidden cell state: \n{}\n".format(i+1, states_[i]))

outputs: shape: (3, 5, 4)
[[[-0.07938521 -0.08045461 -0.11057801 -0.548945  ]
  [-0.3779834  -0.4411604  -0.4776888  -0.51664066]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[-0.003076    0.08089301 -0.62068415 -0.81495845]
  [-0.33286124 -0.59247947 -0.8774056  -0.43668047]
  [-0.7140866  -0.18756755 -0.76010275 -0.8602962 ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[-0.00220012  0.09204233 -0.64357686 -0.8245329 ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]]

Number of Layers: 3
Layer 1 hidden cell state: 
[[-0.9338281   0.93689495  0.2938209   0.99999374]
 [-0.9998581   1.          0.9578909   1.        ]
 [-0.99996024  1.          0.9839632 

### Multi-Layer RNN with basic LSTM Cell

In [13]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('MultiRNN_LSTM', reuse=tf.AUTO_REUSE) as scope:
    # MultiLayer RNN
    def lstm_cell(hidden_size):
      cell = rnn.BasicLSTMCell(
          num_units=hidden_size, state_is_tuple=True)
      return cell

    num_layers = 3
    multi_cells = rnn.MultiRNNCell([lstm_cell(4) for _ in range(num_layers)],
                                   state_is_tuple=True)
    outputs, states = tf.nn.dynamic_rnn(multi_cells, x_data,
                                        sequence_length=[2, 3, 1],
                                        dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, states_ = sess.run([outputs, states])
    print("outputs: shape: {}\n{}\n".format(outputs_.shape, outputs_))
    print("Number of Layers: {}".format(len(states_))) 
    for i in range(num_layers):
      print("Layer {} memory cell state: \n{}\n".format(i+1, states_[i].c))
      print("Layer {} hidden cell state: \n{}\n".format(i+1, states_[i].h))

outputs: shape: (3, 5, 4)
[[[ 2.0920197e-03  6.5658655e-04 -8.4803137e-04  2.3442516e-03]
  [ 6.3524595e-03  3.9691618e-03 -3.4972571e-04  8.2148211e-03]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]

 [[ 1.4048272e-04 -1.5914498e-03 -1.6979526e-03  8.0133528e-03]
  [-9.6788845e-04 -4.4106226e-03 -3.9039277e-03  2.2062376e-02]
  [-2.7877118e-03 -6.8882983e-03 -5.3026089e-03  3.9211988e-02]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]

 [[ 8.1796010e-05 -1.6882729e-03 -1.7611167e-03  8.1220046e-03]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]]

Number 